## Feed Axes in Machine Tools

In automatised production, feed drives generate the contour of workpieces according to specified movement instructions in conjunction with tools. The control unit of the machine analyses the corresponding signals and relays them via the interpolator to the drive unit as reference variables. It is the actuator mechanism for relative motions between the tool and workpiece. Then, its motor performs a change in rotation angle at the rotational speed specified. This change is converted by mechanical transmission elements to a corresponding position change of the machine component to move. 

To perform a motion specified externally between workpiece and tool, movements of individual axes are superimposed. Each of the individual axes form a closed system comprising drive, power conversion and position determination. The basis of its structural composition is a general cause and effect relationship between system components that can be represented with a control loop.

Control loops define both the information flow as well as the interfaces between internal and external elements of the axis system. For machine tool feed drives, attainable speed, maximum acceleration, attainable feed forces, kinematic accuracy, static and dynamic stiffness are typical parameters of system behavior.

```mermaid
graph LR
    A[Reference variables: Interpolator] -->|Reference| B(Controller)
    B --> C(Converter)
    C --> D(Motor)
    D --> E(Mechanical transmission elements)
    E -->|Process| F[Disturbances: Process]
    F --> E
    F -->|Actual| B
    F -->|Actual| G(Measuring systems)
    G -->|Actual| B
    G -->|Actual| C
    G -->|Actual| D
    
    style A fill:#0000,stroke:#000000,stroke-width:2px
    style B fill:#1E90FF,stroke:#000000,stroke-width:2px
    style C fill:#1E90FF,stroke:#000000,stroke-width:2px
    style D fill:#1E90FF,stroke:#000000,stroke-width:2px
    style E fill:#1E90FF,stroke:#000000,stroke-width:2px
    style F fill:#0000,stroke:#000000,stroke-width:2px
    style G fill:#1E90FF,stroke:#000000,stroke-width:2px


## Motor Position Control

In [ ]:
#include <SimpleFOC.h>

// motor instance
BLDCMotor motor = BLDCMotor(11);
// driver instance
BLDCDriver3PWM driver = BLDCDriver3PWM(9, 10, 11, 8);

// encoder instance
Encoder encoder = Encoder(2, 3, 500);
// channel A and B callbacks
void doA(){encoder.handleA();}
void doB(){encoder.handleB();}

void setup() {
  
  // initialize encoder sensor hardware
  encoder.init();
  encoder.enableInterrupts(doA, doB); 
  // link the motor to the sensor
  motor.linkSensor(&encoder);

  // driver config
  driver.init();
  motor.linkDriver(&driver);

  // set motion control loop to be used
  motor.controller = MotionControlType::angle;

  // controller configuration 
  // default parameters in defaults.h

  // controller configuration based on the control type 
  // velocity PID controller parameters
  // default P=0.5 I = 10 D =0
  motor.PID_velocity.P = 0.2;
  motor.PID_velocity.I = 20;
  motor.PID_velocity.D = 0.001;
  // jerk control using voltage voltage ramp
  // default value is 300 volts per sec  ~ 0.3V per millisecond
  motor.PID_velocity.output_ramp = 1000;

  // velocity low pass filtering
  // default 5ms - try different values to see what is the best. 
  // the lower the less filtered
  motor.LPF_velocity.Tf = 0.01;

  // angle P controller -  default P=20
  motor.P_angle.P = 20;

  //  maximal velocity of the position control
  // default 20
  motor.velocity_limit = 4;
  // default voltage_power_supply
  motor.voltage_limit = 10;

  // use monitoring with serial 
  Serial.begin(115200);
  // comment out if not needed
  motor.useMonitoring(Serial);
  
  // initialize motor
  motor.init();
  // align encoder and start FOC
  motor.initFOC();


  Serial.println("Motor ready.");
  _delay(1000);
}

// angle set point variable
float target_angle = 1;
// timestamp for changing direction
long timestamp_us = _micros();

void loop() {

  // each one second
  if(_micros() - timestamp_us > 1e6) {
      timestamp_us = _micros();
      // inverse angle
      target_angle = -target_angle;   
  }

  // main FOC algorithm function
  motor.loopFOC();

  // Motion control function
  motor.move(target_angle);
}